In [3]:
# !pip install darts
# !pip install -U kaleido

## **Step 1: Import all the libraries and modules**

In [9]:
import pandas as pd
import numpy as np
import os
import sys

%load_ext autoreload
%autoreload 2

target_dir = os.path.abspath(os.path.join(os.getcwd(), '..' ))
sys.path.append(target_dir)
          
from Models.RNNLSTMModel.preprocessing import split_train_valid_test
from Models.RNNLSTMModel.preprocessing import transformed_ts
from Models.RNNLSTMModel.RNNLSTMMODEL import RNNLSTMModel
from Models.RNNLSTMModel.evaluation import evaluation_of_model_RnnLstm_Model    

from Models.ARIMAModel.preprocessing import preprocess_df_to_ts

from sklearn.model_selection import  train_test_split
import logging
import gc

import sys
import warnings
warnings.filterwarnings("ignore")
logging.disable(logging.CRITICAL)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## **Step 2 : Reading Data Daily Traffic Accident Forecasting**

In [10]:
daily_traffic_accident_data_path = '../data/processed/daily_data.csv'
daily_accident_df = pd.read_csv(daily_traffic_accident_data_path)
daily_accident_df

,daily,daily_accident
0,2018-01-01,234
1,2018-01-02,408
2,2018-01-03,231
3,2018-01-04,225
4,2018-01-05,342
...,...,...
1821,2022-12-27,183
1822,2022-12-28,177
1823,2022-12-29,191
1824,2022-12-30,193


## **Step 3 : Split the Data into train and test data**

In [11]:
processed_train_df, processed_test_df = train_test_split(
        daily_accident_df, test_size=0.245, random_state=42, shuffle=False)

## **Step 4 : Preprocess the data into Time series object**

In [12]:
print('[INFO] Converting data into times series types')
ts_train = preprocess_df_to_ts(processed_train_df,'daily','daily_accident','1D')
ts_test = preprocess_df_to_ts(processed_test_df,'daily','daily_accident','1D')

[INFO] Converting data into times series types


## **Step 5 : Split the data into train and validation**

In [13]:
train_subset, val_subset = split_train_valid_test(ts_train, split_point=0.9)

## **Step 6 : Scale & Normalize the data before fitting the RNN_LSTM Model**

In [14]:
print('[INFO] Preprocessing Data for RNN-LSTM Model')
(
transformer_object,
train_ts_transformed,
val_ts_transformed,
series_ts_transformed,  
) = transformed_ts(ts_train, 
                val_subset, 
                train_subset, 
            train_data_status=True)

[INFO] Preprocessing Data for RNN-LSTM Model


## **Step 7 : Training RNNLSTM Model**

In [15]:
rnn_lstm_model = RNNLSTMModel(train_ts_transformed, val_ts_transformed)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

## **Step 8 : Evaluation of the RNNLSTM Model**

In [16]:
print('[INFO] Evaluation of RNN-LSTM Model')
# Example usage
model_name = 'RNNLSTMForecasting'
transformer_ob = transformer_object  # Assuming this is defined elsewhere
evaluation_rnn_lstm_df = evaluation_of_model_RnnLstm_Model(model_name,
                                                transformer_ob=transformer_ob,
                                                ts_test=ts_test,
                                                ts_train=ts_train)
gc.collect()


[INFO] Evaluation of RNN-LSTM Model
window_size :30
Evaluation of input window : 30 & Horizon : 418


Predicting: |          | 0/? [00:00<?, ?it/s]

metrics: {'MAE': 43.25, 'RMSE': 55.41, 'MAPE': 16.79, 'MSE': 3070.7}
window_size :60
Evaluation of input window : 60 & Horizon : 388


Predicting: |          | 0/? [00:00<?, ?it/s]

metrics: {'MAE': 43.65, 'RMSE': 55.77, 'MAPE': 17.07, 'MSE': 3110.02}
window_size :90
Evaluation of input window : 90 & Horizon : 358


...

Predicting: |          | 0/? [00:00<?, ?it/s]

metrics: {'MAE': 41.65, 'RMSE': 52.75, 'MAPE': 15.83, 'MSE': 2782.79}
window_size :150
Evaluation of input window : 150 & Horizon : 298


Predicting: |          | 0/? [00:00<?, ?it/s]

metrics: {'MAE': 41.52, 'RMSE': 52.67, 'MAPE': 15.53, 'MSE': 2774.17}


427

## **Step 9 : Saving the Statistical Results**

In [17]:
print('[INFO] Generating Statistical Results of RNN-LSTM Model')
statistical_result_save_path = os.path.join("../StatisticalResults")
os.makedirs(statistical_result_save_path,exist_ok=True)

evaluation_rnn_lstm_df.to_csv(
f"{statistical_result_save_path}/rnn_lstm_evaluation_results.csv", index=False
)

[INFO] Generating Statistical Results of RNN-LSTM Model
